# 1. Iris 데이터셋을 활용해 클래스별 변수 평균 차이를 검정

## 1. Iris 데이터셋 불러오고, 구조 확인

In [ ]:
import seaborn

iris = seaborn.load_dataset("iris")

print(iris.head())
print()
print(iris.tail())
print()
print(iris.info())

## 2. 기술통계량 산출
### Species별 Petal Length의 평균, 표준편차, 최소값, 최대값, 사분위수 등을 출력하고 그룹별 데이터 개수를 확인

In [ ]:
import pandas

moong_taeng_e = iris.groupby("species")["petal_length"]

print("Maximum")
print(moong_taeng_e.max())
print("\nMinimum")
print(moong_taeng_e.min())
print("\nMean")
print(moong_taeng_e.mean())
print("\nSigma")
print(moong_taeng_e.std())
print("\nQuartile")
print(moong_taeng_e.agg([("Q1", lambda x: x.quantile(0.25)), ("Q2", "median"), ("Q3", lambda x: x.quantile(0.75))]))

## 3. 시각화
### Species별 Petal Length의 분포를 Boxplot으로 시각화하시오.
### Boxplot 아래에 간단한 해석 코멘트를 다시오. (예: 어느 그룹이 평균이 높은지)

In [ ]:
from matplotlib import pyplot

seaborn.boxplot(x="species", y="petal_length", data=iris)
pyplot.show()
# setosa < versicolor < virginica

## 4. 정규성 검정
### Species별로 Shapiro-Wilk 검정을 실시하시오.(해당 검정에 대한 가설 수립 포함)
### 각 그룹의 p-value를 출력하고, 0.05를 기준으로 해석하시오.\

In [ ]:
from scipy.stats import shapiro

for species, mte in moong_taeng_e:
    stat, p = shapiro(mte)
    print(f"p-value of {species}: {p:.6f}")


## 5. 등분산성 검정
### Levene 검정을 통해 3개 그룹 간 등분산성을 검정하시오. (해당 검정에 대한 가설 수립 포함)
### p-value를 출력하고, 0.05를 기준으로 해석하시오.

In [ ]:
from scipy.stats import levene

setosa = iris[iris["species"] == "setosa"]["petal_length"]
versicolor = iris[iris["species"] == "versicolor"]["petal_length"]
virginica = iris[iris["species"] == "virginica"]["petal_length"]
stat, p = levene(setosa, versicolor, virginica)
print(f"p-value == {p:.6f}")

## 6. 가설 수립
H0: 3개의 species에 대해 petal_length의 평균이 모두 같다.  
H1: 3개 중 적어도 하나는 petal_length의 평균이 다르다.

## 7. ANOVA 실행
### One-way ANOVA를 실시하시오.
### ANOVA 표를 출력하고 F값과 p-value를 확인한 후, 유의수준 0.05 기준으로
### 귀무가설 기각 여부를 쓰시오.

In [ ]:
from scipy.stats import f_oneway

# Welch ANOVA (등분산 가정 없이)
stat, p = f_oneway(setosa, versicolor, virginica)
print(f"p-value == {p:.6f}")

## 8. 사후검정 (Tukey HSD)
### ANOVA 결과 유의하다면 Tukey HSD 사후검정을 실시하시오.
### 어떤 쌍 간에 유의미한 차이가 있는지 요약하시오.

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

print(pairwise_tukeyhsd(endog=iris["petal_length"], groups=iris["species"], alpha=0.05))

# 2. 실제 신용카드 사기 데이터셋을 활용해 클래스 불균형 상황에서 분류 모델을 학습

## 1. 데이터 로드 및 기본 탐색
### creditcard.csv를 불러오고 데이터 구조를 확인하시오.
### 정상 거래와 사기 거래 건수를 확인하시오.

In [9]:
import pandas

card_data = pandas.read_csv("creditcard.csv")
print(card_data.head())
print(card_data.info())
print(card_data.describe())
print(card_data["Class"].value_counts())

   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

## 2. 샘플링
### 사기 거래(Class=1)는 전부 유지하고, 정상 거래(Class=0)는 10,000건만 무작위
### 샘플링하시오.(sampling 진행시, random_state는 42로 설정)
### 두 데이터셋을 합쳐 새로운 분석용 데이터프레임으로 만드시오.
### 샘플링 후 Class 비율을 다시 출력하시오.

In [12]:
scam_data = card_data[card_data["Class"] == 1]
normal_data = card_data[card_data["Class"] == 0].sample(n=10000, random_state=42)
new_data = pandas.concat([scam_data, normal_data])
print(new_data["Class"].value_counts())

Class
0    10000
1      492
Name: count, dtype: int64


## 3. 데이터 전처리
### Amount 변수만 표준화(StandardScaler) 하여 새로운 변수 Amount_Scaled로
### 대체하시오. Amount 원본 변수는 제거하시오.
### 그리고 X, y로 데이터프레임을 분리하시오.

In [17]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
new_data["Amount_Scaled"] = scaler.fit_transform(new_data[["Amount"]])
new_data.drop(columns=["Amount"], inplace=True)
X = new_data.drop(columns=["Class"])
y = new_data["Class"]

## 4. 학습 데이터와 테스트 데이터 분할
### train_test_split을 사용해 학습셋:테스트셋 비율을 8:2로 나누고,
### stratify=y 옵션으로 클래스 비율 유지, 분할된 데이터의 Class 비율을 출력하시오.
### (random_state는 42로 설정)

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("train data ratio:", y_train.value_counts(normalize=True))
print(" test data ratio:", y_test.value_counts(normalize=True))

train data ratio: Class
0    0.953056
1    0.046944
Name: proportion, dtype: float64
 test data ratio: Class
0    0.953311
1    0.046689
Name: proportion, dtype: float64


## 5. SMOTE 적용
### 학습 데이터(X_train)에 SMOTE를 적용하여 소수 클래스(사기 거래)를
### 오버샘플링하시오. (왜 SMOTE를 적용해야하는지까지 서술하시오.)
### SMOTE 적용 전후의 사기 거래 건수를 출력하시오.

In [22]:
from imblearn.over_sampling import SMOTE
from collections import Counter

smote = SMOTE(random_state=42)
print("Before SMOTE", Counter(y_train))
Xt_smote, yt_smote = smote.fit_resample(X_train, y_train)
print("After SMOTE", Counter(yt_smote))

Before SMOTE Counter({0: 7999, 1: 394})
After SMOTE Counter({0: 7999, 1: 7999})


## 6. 모델 학습
### 적합한 ML 모델을 선정하여 모델을 학습시키고,
### 테스트셋에서 예측값(predict)과 예측 확률(predict_proba)을 출력하시오.
### classification_report로 Precision, Recall, F1-score를 확인하시오.
### 그리고 average_precision_score로 PR-AUC를 계산하여 출력하시오.

In [ ]:
import numpy
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, average_precision_score

model = LogisticRegression(max_iter=10000, random_state=42)
model.fit(Xt_smote, yt_smote)
prediction = model.predict(X_test)
print(classification_report(y_test, prediction))
prob = model.predict_proba(X_test)[:, 1]
print(average_precision_score(y_test, prob))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      2001
           1       0.82      0.93      0.87        98

    accuracy                           0.99      2099
   macro avg       0.91      0.96      0.93      2099
weighted avg       0.99      0.99      0.99      2099

0.9550235754867127


## 7. 최종 성능 평가
### 모델 선정, 하이퍼파라미터 튜닝과 Threshold 조정 등을 통해 최종 모델이
### 목표 Recall ≥ 0.80, F1 ≥ 0.88, PR-AUC ≥ 0.90 을 달성하였는지
### 여부를 작성하시오.(Class 0, 1 둘 다!)
### 달성하지 못했다면 추가로 어떤 방법을 시도할지 간략히 제안하시오.

In [30]:
print(average_precision_score(1 - y_test, 1 - prob))

0.9988602046601736
